# Data Exploration

In [1]:
%pip install -r requirements.txt

  Obtaining dependency information for ydata-profiling from https://files.pythonhosted.org/packages/a2/19/c58a6c22a04806c70802518eaf70866e5633fe47e671ded361295dfaf839/ydata_profiling-4.6.4-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 1.8 MB/s eta 0:00:002.1 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for phik<0.13,>=0.11.1 from https://files.pythonhosted.org/packages/02/7d/83dcf9ce8865a2ecf93d17c31652f43139d0fa0d41e5e09a50a301470c4e/phik-0.12.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for multimethod<2,>=1.4 from https://files.pythonhosted.org/packages/7f/bd/750245e47e7f307d9f94d4fa84727f4ed9956005dfa671d58be1d531a0f6/multimethod-1.10-py3-none-any.whl.metadata
  Obtaining dependency information for typeguard<5,>=4.1.2 from https://files.pythonhosted.org/packages/18/01/5fc45558268ced46d86292763477996a3cdd505567cd590a688e8cdc386e/typegu

In [5]:
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, make_scorer, classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from config import val_split_seed


ImportError: cannot import name 'val_split_seed' from 'config' (/home/daniel/anaconda3/envs/base_ml/lib/python3.8/site-packages/config/__init__.py)

In [ ]:
a = pd.read_csv("./BankA.csv")
b = pd.read_csv("./BankB.csv")
c = pd.read_csv("./BankC.csv")

In [ ]:
a_samp = a.sample(n=100)
b_samp = b.sample(n=100)
c_samp = c.sample(n=100)

# a_b_samp = a_samp.append(b_samp, ignore_index=True)
# all_samp = a_b_samp.append(c_samp, ignore_index=True)

all_samp = pd.concat([a_samp, b_samp, c_samp], ignore_index=True)

In [ ]:
sns.pairplot(all_samp, height=3);

In [ ]:
profile = ProfileReport(all_samp, title="Profiling Report")
profile.to_notebook_iframe()

In [ ]:
all_samp.head(3)

In [ ]:
# Native Country (Rassistisch?)
# Gender (sexismus)
# Race (Rassistisch!)
# Relationship (Husband, Wife einfach zu Married?)
# Educational-num oder education weg, je nach dem ob one hot encoding oder nicht

In [ ]:
df = pd.concat([a, b, c], ignore_index=True)

In [ ]:
drop_this = ["institute","education","race","gender","native-country"]
df = df.drop(drop_this, axis = 1)

In [ ]:
df.loc[df["relationship"].isin(["Husband", "Wife"]), "relationship"] = "Married"

In [ ]:
df.dtypes

In [ ]:
df = pd.get_dummies(df, columns=["workclass", "marital-status", "occupation", "relationship"])

In [ ]:
df.head()

In [ ]:
y = df['income']
X = df.drop('income', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=val_split_seed)

In [ ]:
# Modelle für Klassifikation
models = {'Logistic Regression': LogisticRegression(),
          'Decision Tree': DecisionTreeClassifier(random_state=42),
          'Random Forest': RandomForestClassifier(random_state=42),
          'Gradient Boosting': GradientBoostingClassifier(random_state=42)}

# Evaluation Metriken für Klassifikation
results = {'Model': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1 Score': [], 'Cohen\'s Kappa': []}

# Modellentwicklung und Evaluation
for model_name, model in models.items():
    # Modell anpassen
    model.fit(X_train, y_train)
    
    # Vorhersagen
    predictions = model.predict(X_test)
    
    # Metriken
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, pos_label='>50K')
    recall = recall_score(y_test, predictions, pos_label='>50K')
    f1 = f1_score(y_test, predictions, pos_label='>50K')
    kappa = cohen_kappa_score(y_test, predictions)
    
    # Ergebnisse speichern
    results['Model'].append(model_name)
    results['Accuracy'].append(accuracy)
    results['Precision'].append(precision)
    results['Recall'].append(recall)
    results['F1 Score'].append(f1)
    results['Cohen\'s Kappa'].append(kappa)
    
    # Metriken anzeigen
    print("Klassifikationsbericht:\n", classification_report(y_test, predictions))
    print(f"{model_name}\nAccuracy: {accuracy:.2f}\nPrecision: {precision:.2f}\nRecall: {recall:.2f}\nF1 Score: {f1:.2f}\nCohen's Kappa: {kappa:.2f}\n{'='*40}")

# Ergebnisse in ein DataFrame konvertieren, um sie besser zu visualisieren
pd.DataFrame(results).sort_values(by='Accuracy', ascending=False)

In [ ]:
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
y = df['income']
X = df.drop('income', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Für binäre Klassifikation

# Kompilieren des Modells
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Konvertieren der Daten in Tensoren (sollte das auf 0-1 skaliert sein?)
X_train_tensor = tf.convert_to_tensor(np.asarray(X_train).astype('float32'))
y_train_tensor = tf.convert_to_tensor(np.asarray(y_train).astype('float32'))
X_test_tensor = tf.convert_to_tensor(np.asarray(X_test).astype('float32'))
y_test_tensor = tf.convert_to_tensor(np.asarray(y_test).astype('float32'))

# Trainieren des Modells
model.fit(X_train_tensor, y_train_tensor, epochs=10, batch_size=32)

# Bewertung des Modells
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)

results['Model'].append('Neural Network')
results['Accuracy'].append(accuracy)
results['Precision'].append(np.nan)
results['Recall'].append(np.nan)
results['F1 Score'].append(np.nan)
results['Cohen\'s Kappa'].append(np.nan)
pd.DataFrame(results).sort_values(by='Accuracy', ascending=False)

# Train Models for each Bank separately

In [ ]:

# Load BankA.csv
bankA_df = pd.read_csv('BankA.csv')
print(bankA_df.shape)

# Load BankB.csv
bankB_df = pd.read_csv('BankB.csv')
print(bankB_df.shape)

# Load BankC.csv
bankC_df = pd.read_csv('BankC.csv')
print(bankC_df.shape)


In [ ]:
np.random.seed(42) 

In [ ]:
bankA_df.head(3)

In [ ]:
# Define function to split data into train and test sets

def split_data(df):
    # Drop "race" and "gender" columns
    df_processed = df.drop(['race', 'gender'], axis=1)
    
    # Encode categorical variables
    label_encoders = {}
    for column in df_processed.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        df_processed[column] = label_encoders[column].fit_transform(df_processed[column])
    
    # Split data into features and target variable
    X = df_processed.drop('income', axis=1)
    y = df_processed['income']
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features using MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test


In [ ]:
# Define function to train logistic regression model
def train_model(X_train, X_test, y_train, y_test, model):
    # Train model
    model.fit(X_train, y_train)
    
    # Evaluate model
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
    return model, train_score, test_score


In [ ]:
# Split data for Bank A
X_trainA, X_testA, y_trainA, y_testA = split_data(bankA_df)

# Split data for Bank B
X_trainB, X_testB, y_trainB, y_testB = split_data(bankB_df)

# Split data for Bank C
X_trainC, X_testC, y_trainC, y_testC = split_data(bankC_df)


In [ ]:
LogReg = LogisticRegression(max_iter=1000)

# Train logistic regression models for each dataframe
bankA_model_log, bankA_train_score_log, bankA_test_score_log = train_model(X_trainA, X_testA, y_trainA, y_testA, model=LogReg)
bankB_model_log, bankB_train_score_log, bankB_test_score_log = train_model(X_trainB, X_testB, y_trainB, y_testB, model=LogReg)
bankC_model_log, bankC_train_score_log, bankC_test_score_log = train_model(X_trainC, X_testC, y_trainC, y_testC, model=LogReg)

# Display training and testing scores for each model
print("Bank A - Training Score:", bankA_train_score_log)
print("Bank A - Testing Score:", bankA_test_score_log)
print("Bank B - Training Score:", bankB_train_score_log)
print("Bank B - Testing Score:", bankB_test_score_log)
print("Bank C - Training Score:", bankC_train_score_log)
print("Bank C - Testing Score:", bankC_test_score_log)



In [ ]:
from sklearn.svm import SVC

# Create SVC model
svm_model = SVC()

# Train SVM models for each dataframe
bankA_model_svm, bankA_train_score_svm, bankA_test_score_svm = train_model(X_trainA[:len(X_trainA)//10], X_testA, y_trainA[:len(X_trainA)//10], y_testA, model=svm_model)
print('BankA Training finished')
bankB_model_svm, bankB_train_score_svm, bankB_test_score_svm = train_model(X_trainB[:len(X_trainB)//10], X_testB, y_trainB[:len(X_trainB)//10], y_testB, model=svm_model)
print('BankB Training finished')
bankC_model_svm, bankC_train_score_svm, bankC_test_score_svm = train_model(X_trainC[:len(X_trainC)//10], X_testC, y_trainC[:len(X_trainC)//10], y_testC, model=svm_model)
print('BankC Training finished')

# Display training and testing scores for each model
print("Bank A - Training Score:", bankA_train_score_svm)
print("Bank A - Testing Score:", bankA_test_score_svm)
print("Bank B - Training Score:", bankB_train_score_svm)
print("Bank B - Testing Score:", bankB_test_score_svm)
print("Bank C - Training Score:", bankC_train_score_svm)
print("Bank C - Testing Score:", bankC_test_score_svm)